# Analiza: Wykrywanie języka na podstawie częstotliwości słów w tekscie

Celem tej pracy jest ocena, czy na podstawie danych dotyczących częstotliwości występowania słów w arytkule na wybranym wiki (u nas bulbapedia) da się rozpoznać język tekstu. Eksperyment polega na porównaniu listy najczęsciej występujących słów na wiki z listą najczęściej występujących słów w jednym z trzech języków. Sprawdzimy dodatkowo jak na poprawność metody wpływa długość arytkułu.

In [2]:
# Początkowe importy najważniejszych modułów i bibliotek
import json
import re
import matplotlib.pyplot as plt
from wordfreq import top_n_list, zipf_frequency
from pathlib import Path
from scraper_logic import Scraper
from bs4 import BeautifulSoup
import requests

Wykorzystamy listy najczęsciej występujących słów z trzech języków: polskiego, angielskiego (język naszej wiki), oraz niemieckiego. Użyjemy do tego biblioteki wordfreq.

In [3]:
def get_language_words(lang, n=1000):
    words = top_n_list(lang, n)
    return {
        w : zipf_frequency(w, lang)
        for w in words 
    }
    
en_dict = get_language_words('en')
pl_dict = get_language_words('pl')
de_dict = get_language_words('de')

Do pobrania słów z arytkułów z wiki użyjemy pliku tworzonego wywołaniem funkcji scrapera do_count_words(), która aktualizuje plik o słownik [słowa : liczba wystąpień] dla danej frazy. 

In [4]:
def read_word_file(): 
    with open('./word-counts.json', 'r', encoding='utf-8') as f:
        return json.load(f)
    
def get_words(phrase):
    # Tworzymy instancje scrapera dla danej frazy.
    scraper = Scraper(phrase)

    # Usuwamy zawartość pliku word-count.json
    try:
        Path('./word-counts.json').unlink()
    except FileNotFoundError:
        pass
    
    # Liczymy słowa dla danego artykułu
    scraper.do_count_words()

    # Zwracamy otrzymany słownik
    return read_word_file()

Przygotujmy i zapiszmy do zmiennej dane z długiego artykułu. Tworzymy dedykowany plik word-analysis.json, używamy scrapera do zapełnienia go słowami z danego artykułu i zapisujemy z użyciem powyższej funkcji. Jako długi arytykuł użyjemy opisu regionu Paldea z bulbapedii [https://bulbapedia.bulbagarden.net/wiki/Paldea].

In [5]:
long_article = get_words('Paldea')

Zrobimy to samo dla krótkiego arytkuły ze strony [https://bulbapedia.bulbagarden.net/wiki/Bulbakaki], który pełen jest nazw własnych.

In [6]:
short_article = get_words('Bulbakaki')

Musimy zrobić to samo dla artykułów w każdym z wybranych języków do czego będą nam potrzebne nowe funkcje, zacznijmy od pozyskania kodu html strony w formie obiektu soup:

In [35]:
def get_article(url):
    # Tworzymy header, który pozwoli nam się dostać do stron używając headera z httpbin.org/cache

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:147.0) Gecko/20100101 Firefox/147.0'
    }

    try:
        source = requests.get(url, headers=headers)
    except requests.RequestException:
        return None

    if source.status_code != 200:
        print(f"Błąd: status_code={source.status_code} dla {url}")
        return None

    return BeautifulSoup(source.text, 'html.parser')

pl_soup = get_article('https://wolnelektury.pl/katalog/lektura/witkacy-o-czystej-formie.html')
en_soup = get_article('https://minecraft.fandom.com/wiki/Trading')
de_soup = get_article('https://wolnelektury.pl/katalog/lektura/heyse-die-schwarze-jakobe.html')

# Usuwamy polskie słowa z niemieckiego tekstu
for h2 in de_soup.find_all("h2"):
    h2.decompose()

Zdefiniujmy funkcję do liczenia słów:

In [28]:
def count_words(text):
    word_dict = {}

    for word in text:
        word_dict[word] = word_dict.get(word, 0) + 1

    return word_dict


Teraz musimy w każdej z tych struktur znaleźć właściwą zawartość z artykułem, zamienić ją na tekst i policzyć słowa:

In [ ]:
# Znajdowanie właściwej zawartości (find zwraca typ tag)
pl_content = pl_soup.find('div', class_='main-text-body')
en_content = en_soup.find('div', id='content')
de_content = de_soup.find('div', class_='main-text-body')

# Zamieniamy typ na string
pl_text = pl_content.get_text(" ", strip=True)
en_text = en_content.get_text(" ", strip=True)
de_text = de_content.get_text(" ", strip=True)

# Parsujemy słowa
words_pl = re.findall(r'\w+', pl_text.lower())
words_en = re.findall(r'\w+', en_text.lower())
words_de = re.findall(r'\w+', de_text.lower())

# Używamy naszej funckji do stworzenia słowników z licznikiem wystąpień
pl_dict = count_words(words_pl)
en_dict = count_words(words_en)
de_dict = count_words(words_de)




[('paul', 1), ('heyse', 1), ('die', 299), ('schwarze', 20), ('jakobe', 14), ('1883', 1), ('1', 1), ('eines', 15), ('abends', 4), ('als', 100), ('ich', 536), ('meiner', 32), ('täglichen', 2), ('gewohnheit', 2), ('nach', 40), ('bei', 29), ('frau', 34), ('von', 87), ('f', 1), ('eintrat', 2), ('fand', 12), ('meine', 47), ('alte', 16), ('freundin', 12), ('nicht', 205), ('wie', 118), ('sonst', 13), ('in', 183), ('ihrem', 18), ('lehnstuhl', 2), ('am', 38), ('tische', 2), ('sitzend', 1), ('hinter', 7), ('dem', 89), ('grünen', 1), ('lichtschirm', 1), ('dessen', 3), ('schatten', 2), ('sie', 416), ('der', 227), ('vorlesung', 1), ('ihres', 10), ('fräuleins', 1), ('zuzuhören', 3), ('pflegte', 3), ('das', 178), ('buch', 5), ('zwar', 1), ('lag', 8), ('aufgeschlagen', 1), ('neben', 9), ('lampe', 1), ('platz', 2), ('vorleserin', 1), ('aber', 108), ('war', 123), ('leer', 2), ('und', 581), ('dame', 1), ('ging', 8), ('trotz', 9), ('ihrer', 19), ('gebrechlichkeit', 1), ('mit', 124), ('hastigen', 1), ('aufg

Upewnijmy się, że wszystko działa wypisując pare pierwszych słów:

In [40]:
pl_first_items = list(pl_dict.items())[:100]
en_first_items = list(en_dict.items())[:100]
de_first_items = list(de_dict.items())[:100]

print(pl_first_items)
print(en_first_items)
print(de_first_items)

[('spis', 1), ('treści', 15), ('artysta', 5), ('1', 11), ('muzyka', 5), ('2', 7), ('piękno', 8), ('3', 5), ('poezja', 5), ('przemijanie', 2), ('realista', 2), ('sztuka', 15), ('4', 4), ('5', 4), ('teatr', 4), ('informacja', 1), ('o', 61), ('dokonanych', 1), ('zmianach', 1), ('i', 235), ('wprowadzono', 1), ('uwspółcześnienia', 1), ('w', 278), ('następującym', 1), ('zakresie', 1), ('zmiany', 2), ('leksykalne', 1), ('tym', 28), ('ortograficzne', 1), ('np', 24), ('tłómaczy', 1), ('tłumaczy', 3), ('massę', 1), ('masę', 2), ('zadowolnić', 1), ('zadowolić', 2), ('niewytłomaczoną', 1), ('niewytłumaczoną', 2), ('udźwięcznienia', 1), ('ubezdźwięcznienia', 1), ('zcałkowanie', 1), ('scałkowanie', 2), ('pisownia', 2), ('łączna', 1), ('rozdzielna', 1), ('któryby', 1), ('który', 8), ('by', 6), ('naprzykład', 1), ('na', 101), ('przykład', 3), ('niewiadomo', 1), ('nie', 120), ('wiadomo', 4), ('napewno', 1), ('pewno', 2), ('będący', 1), ('niebędący', 1), ('mający', 2), ('niemający', 1), ('joty', 1), ('t